# Load libraries

In [ ]:
!pip install wikiextractor blingfire

import gensim
from gensim.models import Word2Vec
import os 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.1 MB 1.8 MB/s 


# Unzip wiki dumps (takes a while)

Hint: change the directory!

In [ ]:
!wget https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2

--2022-12-15 12:50:58--  https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810800523 (773M) [application/octet-stream]
Saving to: ‘idwiki-latest-pages-articles.xml.bz2’

idwiki-latest-pages 100%[===================>] 773.24M  4.11MB/s    in 3m 8s   

2022-12-15 12:54:06 (4.12 MB/s) - ‘idwiki-latest-pages-articles.xml.bz2’ saved [810800523/810800523]



In [ ]:
!bzip2 -d idwiki-latest-pages-articles.xml.bz2

In [ ]:
!sh extract_and_clean_wiki_dump.sh idwiki-latest-pages-articles.xml

Cloning into 'wikiextractor'...
remote: Enumerating objects: 766, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 766 (delta 13), reused 19 (delta 10), pack-reused 741
Receiving objects: 100% (766/766), 1.31 MiB | 23.98 MiB/s, done.
Resolving deltas: 100% (446/446), done.
Extracting and cleaning idwiki-latest-pages-articles.xml to idwiki-latest-pages-articles.txt...
Succesfully extracted and cleaned idwiki-latest-pages-articles.xml to idwiki-latest-pages-articles.txt


In [ ]:
!python preprocess_wiki_dump.py idwiki-latest-pages-articles.txt

Pre-processing idwiki-latest-pages-articles.txt to idwiki-latest-pages-articles_preprocessed.txt...
Successfully pre-processed idwiki-latest-pages-articles.txt to idwiki-latest-pages-articles_preprocessed.txt...


In [ ]:
#!python train_embeddings.py tlwiki-latest-pages-articles.txt es_model.model

# Scrape panlex


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize


def scrape_panlex_tables(URL, pages=1, columns=['Source', 'Target']):
  table = pd.DataFrame([])
  for i in range(pages):
    scraped_tables = pd.read_html(URL+str(i),)
    if scraped_tables:
      table = pd.concat([table, scraped_tables[0]], axis=0)
  return table.rename(columns={'Unnamed: 0':'Source', 'Unnamed: 1': 'Target'})    


URL = "https://vocab.panlex.org/tgl-000/eng-000?page="


table = scrape_panlex_tables(URL, 300, columns=['EN', 'TL'])

In [57]:
table_MN = table.drop(['EN', 'TL'], axis=1).dropna().drop_duplicates()
table_MN = table_MN[table_MN['Source'] != table_MN['Target']]
table_MN = table_MN[~table_MN['Source'].str.contains(' ')]
table_MN = table_MN[~table_MN['Target'].str.contains(' ')]
table_MN = table_MN[~((table_MN['Source']=='Tagalog') & (table_MN['Target']=='English'))]
table_MN.to_csv('tl_en_dict.txt', sep=' ',index=False, header=False)

In [59]:
table_MN[['Target', 'Source']].to_csv('tl_en_dict.txt', sep=' ',index=False, header=False)

,Source,Target
44,english,English
22,Inglatero,English
24,ingles,English
